In [1]:
# Install dependencies for this example
# Note: This does not include itk-jupyter-widgets, itself
import sys
!{sys.executable} -m pip install itk-io requests tqdm

You are using pip version 9.0.2, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import requests
from tqdm import tqdm_notebook as tqdm
import os

import numpy as np
import itk

from itkwidgets import view
import itkwidgets

In [5]:
# Download example data
# Source: https://cvlab.epfl.ch/data/em
#
# 1024x768x165 unsigned char, 124MB file
url = 'https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/training.tif'
# 2048x1536x1065 unsigned char, 3.2GB file
#url = 'https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/volumedata.tif'
filename = 'hippocampus.tif'

if not os.path.exists(filename):
    chunk_size = 32 * 1024
    r = requests.get(url, stream=True)
    total_size = int(r.headers.get('content-length', 0))
    pbar = tqdm(unit="B", unit_scale=True, total=int(total_size))
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=chunk_size): 
            if chunk: # filter out keep-alive new chunks
                pbar.update(len(chunk))
                f.write(chunk)

In [8]:
image = itk.imread(filename)
%time view(image, gradient_opacity=0.3, cmap=itkwidgets.cm.gray, slicing_planes=True, annotations=False)

CPU times: user 584 ms, sys: 0 ns, total: 584 ms
Wall time: 98 ms


Viewer(annotations=False, cmap=u'gray_Matlab', gradient_opacity=0.3, rendered_image=<itkImagePython.itkImageUC3; proxy of <Swig Object of type 'itkImageUC3 *' at 0x7fa90870a1e0> >, roi=[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], size_limit_2d=[2048, 2048], size_limit_3d=[256, 256, 256], slicing_planes=True)

Next, click the crop button in the widget user interface. Adjust the location of the region of interest (roi) by click and dragging the corner of the cropping box.

In [6]:
# Get the selected region as an itk.ImageRegion
shrnker = itk.BinShrinkImageFilter.New(image)
shrnker.GetShrinkFactors()

itkFixedArrayUI3 ([1, 1, 1])

In [3]:
# Get the selected region as NumPy array slices
1024 / 513

1

In [18]:
# Use the slices to extract the region of interest
arr = itk.GetArrayFromImage(image)
roi = viewer.roi_slice()
view(arr[roi], gradient_opacity=0.4, cmap=itkwidgets.cm.PrGn, annotations=False, ui_collapsed=True)

Viewer(annotations=False, cmap='PRGn', gradient_opacity=0.4, rendered_image=<itkImagePython.itkImageUS3; proxy…